In [ ]:
from geojson import Point, LineString, Feature, FeatureCollection, dump
import pandas as pd

# load data file
df = pd.read_csv('all_yearly_averages.csv')

# list of toxins
toxins = ['Air Toxics Concentrations- Average Benzene Concentrations',
          'Air Toxics Concentrations- Average Formaldehyde Concentrations',
          'Fine Particulate Matter (PM2.5)',
          'Nitrogen Dioxide (NO2)',
          'Ozone (O3)',
          'Sulfur Dioxide (SO2)']

# loop through borough, year, and toxin
for borough in df['Borough'].unique():
    for year in df['Year'].unique():
        for toxin in toxins:
            # filter for the current borough, year, and toxin
            df_filtered = df[(df['Borough'] == borough) & (df['Year'] == year) & (df['Toxin'] == toxin)]

            # extract information for geojson generation
            entries = []
            for idx, entry in df_filtered.iterrows():
                entries.append(((entry['Longitude'], entry['Latitude']), entry['Data Value']))

            # sort entries in ascending order, by their measured data value
            entries.sort(key=lambda x: x[1])

            # check if there are any entries for this combination of borough, year, and toxin
            if not len(entries) == 0:
                features = []
                # adding points and connecting lines to list of features
                for i in range(len(entries)):
                    point = Point(entries[i][0])
                    features.append(Feature(geometry=point, properties={"measure": entries[i][1]}))

                    if not i == len(entries) - 1:
                        line = LineString([entries[i][0], entries[i + 1][0]])
                        features.append(Feature(geometry=line, properties={"measure": entries[i][1]}))

                feature_collection = FeatureCollection(features)
                with open(f'{borough}, {year} - {toxin}.geojson', 'w') as file:
                    dump(feature_collection, file)